In [2]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, IntegerType, DateType

SCHEMA = StructType([
    StructField("DATAOCORRENCIA", DateType(), True),
    StructField("HORAOCORRENCIA", StringType(), True),
    StructField("PERIDOOCORRENCIA", StringType(), True),
    StructField("LOGRADOURO", StringType(), True),
    StructField("NUMERO", StringType(), True),
    StructField("BAIRRO", StringType(), True),
    StructField("CIDADE", StringType(), True),
    StructField("UF", StringType(), True),
    StructField("LATITUDE", DoubleType(), True),
    StructField("LONGITUDE", DoubleType(), True),
    StructField("DESCRICAOLOCAL", StringType(), True),
    StructField("DELEGACIA_NOME", StringType(), True),
    StructField("DELEGACIA_CIRCUNSCRICAO", StringType(), True),
    StructField("STATUS", StringType(), True),
    StructField("TIPOPESSOA", StringType(), True),
    StructField("VITIMAFATAL", StringType(), True),
    StructField("NATURALIDADE", StringType(), True),
    StructField("SEXO", StringType(), True),
    StructField("DATANASCIMENTO", DateType(), True),
    StructField("IDADE", IntegerType(), True),
    StructField("ESTADOCIVIL", StringType(), True),
    StructField("PROFISSAO", StringType(), True),
    StructField("CORCUTIS", StringType(), True),
    StructField("UF_VEICULO", StringType(), True),
    StructField("CIDADE_VEICULO", StringType(), True),
    StructField("DESCR_COR_VEICULO", StringType(), True),
    StructField("DESCR_MARCA_VEICULO", StringType(), True),
    StructField("ANO_FABRICACAO", IntegerType(), True),
    StructField("ANO_MODELO", IntegerType(), True),
    StructField("DESCR_TIPO_VEICULO", StringType(), True),
    StructField("QUANT_CELULAR", StringType(), True),
    StructField("MARCA_CELULAR", StringType(), True),
    StructField("HORAS", IntegerType(), True),
    StructField("MINUTOS", IntegerType(), True),
    StructField("ANO", IntegerType(), True),
    StructField("MES", IntegerType(), True),
    StructField("DIA_MES", IntegerType(), True),
    StructField("DIA_SEMANA", IntegerType(), True),
    StructField("DIA_ANO", IntegerType(), True),
    StructField("SEMANA_ANO", IntegerType(), True),
    StructField("MARCA_VEICULO", StringType(), True),
    StructField("MODELO_VEICULO", StringType(), True),
])

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, isnan, when, split, regexp_replace, to_date, year, month, dayofmonth, dayofweek, dayofyear, weekofyear, upper, translate, trim
import pandas as pd
import unicodedata
#from schema import SCHEMA

from pyspark.sql import SparkSession
import boto3

In [4]:
# Retrieve AWS credentials from the EC2 instance metadata
credentials = boto3.Session().get_credentials()

# Configure SparkSession with S3 credentials
spark = SparkSession.builder \
    .appName("ImportCarCleanDataToMySQL") \
    .config("spark.hadoop.fs.s3a.access.key", credentials.access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", credentials.secret_key) \
    .config("spark.hadoop.fs.s3a.session.token", credentials.token) \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,mysql:mysql-connector-java:8.0.17") \
    .getOrCreate()

In [5]:
S3 = "s3a://criminix-criminix-clean/car_theft/car_theft_csv"

df = spark.read.option("header", "true").csv(S3, schema=SCHEMA)

df.count()

1601935

In [7]:
df.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://3.211.108.7/criminix") \
  .option("dbtable", "criminix") \
  .option("user", "criminix") \
  .option("password", "criminix") \
  .option("truncate","true") \
  .mode("overwrite") \
  .save()